In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = DirectoryLoader("/home/jinzy/work/automation/ask_gpt/testdocs", glob="**/*.pdf", show_progress=True, use_multithreading=True)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 400,
    chunk_overlap  = 30,
    length_function = len,
)
txt_docs = loader.load_and_split(text_splitter=text_splitter)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.56s/it]


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import GPT4All
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
# llm = HuggingFaceHub(repo_id='THUDM/chatglm-6b', model_kwargs={"trust_remote_code": True})
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10})
# llm = ChatGLMService()
# llm.load_model()

embeddings = OpenAIEmbeddings()
# embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
#                                            model_kwargs={"device": "cuda"})
txt_docsearch = Chroma.from_documents(txt_docs, embeddings)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=2048)
# llm = GPT4All(model="/home/jinzy/Downloads/ggml-gpt4all-j-v1.3-groovy.bin", n_ctx=512, n_threads=8)

In [4]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


rewritte_template = """根据聊天记录和问题，把问题重写为一个独立的、完整的问题。
聊天记录:
{chat_history}
问题: {question}
独立问题:"""

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_template(rewritte_template)
# 初始化问答链
question_generator = LLMChain(llm=llm, prompt=prompt)

refine_template = """请检查下述检索结果是否可以用于更新回复内容。
检索结果:
{context_str}

如果这个检索结果包含了有用的信息，请根据它和原始回复内容重新生成更好的回复。如果没有帮助，就返回原始回复内容。
新的回复："""
# (
#     "We have the opportunity to refine the existing answer"
#     "(only if needed) with some more context below.\n"
#     "------------\n"
#     "{context_str}\n"
#     "------------\n"
#     "Given the new context, refine the original answer to better "
#     "answer the question. "
#     "If the context isn't useful, return the original answer."
# )
refine_prompt = ChatPromptTemplate.from_template(refine_template)
doc_chain = load_qa_chain(llm, chain_type="refine", refine_prompt =refine_prompt)

summary_template = """根据聊天内容增量式生成摘要，在原始摘要上附加新的摘要。
EXAMPLE
当前摘要: 
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

新的聊天内容:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

新的摘要:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

当前摘要:
{summary}

新的聊天内容:
{new_lines}

新的摘要:
"""

summary_prompt = PromptTemplate(input_variables=["summary", "new_lines"], template=summary_template)

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationSummaryBufferMemory(llm=llm, prompt = summary_prompt,
        max_token_limit=1000, memory_key="chat_history", return_messages=True)
chain = ConversationalRetrievalChain(
    retriever=txt_docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory
)
# chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=txt_docsearch.as_retriever(), 
#                                               chain_type='refine', memory=memory)

In [5]:
query = "请用中文解释一下RPM产品中的任务挖掘功能"
result = chain({"question": query})
print(result)

{'question': '请用中文解释一下RPM产品中的任务挖掘功能', 'chat_history': [HumanMessage(content='请用中文解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False), AIMessage(content='iS-RPM是一种数据驱动的智能流程自动化解决方案，可以帮助企业在RPA实现过程中自动、智能地发现值得自动化的业务流程，并通过与RPA的无缝对接实现整个RPA规模化闭环。该解决方案具有成熟的跨平台跨系统数据采集能力，支持跨业务如ERP、OA等系统，同时也支持windows桌面、web平台、虚拟机和云桌面的数据采集。使用iS-RPM收集流程和任务级别的数据，可以代替繁琐的人工业务梳理过程，并提供洞察力，帮助企业管理者快速发现业务瓶颈，并根据不同的标准推荐高价值的自动化机会。', additional_kwargs={}, example=False)], 'answer': 'iS-RPM是一种数据驱动的智能流程自动化解决方案，可以帮助企业在RPA实现过程中自动、智能地发现值得自动化的业务流程，并通过与RPA的无缝对接实现整个RPA规模化闭环。该解决方案具有成熟的跨平台跨系统数据采集能力，支持跨业务如ERP、OA等系统，同时也支持windows桌面、web平台、虚拟机和云桌面的数据采集。使用iS-RPM收集流程和任务级别的数据，可以代替繁琐的人工业务梳理过程，并提供洞察力，帮助企业管理者快速发现业务瓶颈，并根据不同的标准推荐高价值的自动化机会。'}


In [6]:
query = "请给出一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})
#print(result)
print(result['answer'])

如果您足够了解流程业务，可以针对指定步骤添加过滤并判断它们是否真的有必要重复执行，深入研究数据并验证它们。此外，iS-RPM 还支持对比分析两个不同维度的流程性能，通过选择特定的数据属性值，一次性对比查看流程中的两组路径之间的差异，以确定最佳实践并在不同地区、业务部门和团队之间共享它们。一旦您确定了业务流程的最佳实践，即可逐步开展业务流程的相应改进措施。通过案例详情查看所有案例的步骤细节，可以清晰地了解流程之间的距离，知道案例何时开始超过流程的平均执行时长。


In [7]:
query = "请介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})

print(result['answer'])

iS-RPM 是一款流程挖掘技术，可以通过摄取系统日志或事件日志来获取业务过程，并重建流程中发生的事情。分析师可以将系统中的事件日志导出为 .csv 文件并上传到流程挖掘平台中，通过数据建模和流程挖掘算法构建出全量的业务流程图，实现企业级流程监控。

在长期的运营过程中，企业的标准流程可能无法完全适应员工和客户的工作方式，流程在日常的运行过程中也会有变化。iS-RPM 可以通过对业务数据的识别分析出业务的不同执行方式，将每个案例在整个周期中所采取的每一步叠加到数据模型中，还原出100%客观的流程视图，真实直观的展示员工每一次业务执行轨迹。


In [8]:
result

{'question': '请介绍一个RPM产品的实际案例',
 'chat_history': [HumanMessage(content='请用中文解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False),
  AIMessage(content='iS-RPM是一种数据驱动的智能流程自动化解决方案，可以帮助企业在RPA实现过程中自动、智能地发现值得自动化的业务流程，并通过与RPA的无缝对接实现整个RPA规模化闭环。该解决方案具有成熟的跨平台跨系统数据采集能力，支持跨业务如ERP、OA等系统，同时也支持windows桌面、web平台、虚拟机和云桌面的数据采集。使用iS-RPM收集流程和任务级别的数据，可以代替繁琐的人工业务梳理过程，并提供洞察力，帮助企业管理者快速发现业务瓶颈，并根据不同的标准推荐高价值的自动化机会。', additional_kwargs={}, example=False),
  HumanMessage(content='请给出一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='如果您足够了解流程业务，可以针对指定步骤添加过滤并判断它们是否真的有必要重复执行，深入研究数据并验证它们。此外，iS-RPM 还支持对比分析两个不同维度的流程性能，通过选择特定的数据属性值，一次性对比查看流程中的两组路径之间的差异，以确定最佳实践并在不同地区、业务部门和团队之间共享它们。一旦您确定了业务流程的最佳实践，即可逐步开展业务流程的相应改进措施。通过案例详情查看所有案例的步骤细节，可以清晰地了解流程之间的距离，知道案例何时开始超过流程的平均执行时长。', additional_kwargs={}, example=False),
  HumanMessage(content='请介绍一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='iS-RPM 是一款流程挖掘技术，可以通过摄取系统日志或事件日志来获取业务过程，并重建流程中发生的事情。分析师可以将系统中的事件日志导出为 .csv 文件并上传到流程挖掘平台中，通过数

In [9]:
query = "请介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query + ' 请用中文回答。'})

print(result['answer'])

iS-RPM是一个按时间顺序排列的事件序列，显示了案例从流程开始到结束所采取的所有不同路径。每个唯一路径称为一个变体，不遵循标准或公认路径的变体称为偏差。通过多维度的分析，我们可以了解每个环节的时长，定位流程的堵点，从而找到流程优化的切入点。艺赛旗 iS-RPM 的流程发现仪表板支持逐级钻取，通过使用不同的过滤条件（如：步骤筛选、属性筛选等）多维度的流程分析，能够快速了解并定位流程效率低下的根本原因，并量化它们对业务关键性 KPI 指标的影响。在了解流程全貌后，我们可以解决一些业务执行过程中的困惑，例如瓶颈在哪里，订单为什么迟迟未支付等问题。


In [10]:
result

{'question': '请介绍一个RPM产品的实际案例 请用中文回答。',
 'chat_history': [HumanMessage(content='请给出一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='如果您足够了解流程业务，可以针对指定步骤添加过滤并判断它们是否真的有必要重复执行，深入研究数据并验证它们。此外，iS-RPM 还支持对比分析两个不同维度的流程性能，通过选择特定的数据属性值，一次性对比查看流程中的两组路径之间的差异，以确定最佳实践并在不同地区、业务部门和团队之间共享它们。一旦您确定了业务流程的最佳实践，即可逐步开展业务流程的相应改进措施。通过案例详情查看所有案例的步骤细节，可以清晰地了解流程之间的距离，知道案例何时开始超过流程的平均执行时长。', additional_kwargs={}, example=False),
  HumanMessage(content='请介绍一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='iS-RPM 是一款流程挖掘技术，可以通过摄取系统日志或事件日志来获取业务过程，并重建流程中发生的事情。分析师可以将系统中的事件日志导出为 .csv 文件并上传到流程挖掘平台中，通过数据建模和流程挖掘算法构建出全量的业务流程图，实现企业级流程监控。\n\n在长期的运营过程中，企业的标准流程可能无法完全适应员工和客户的工作方式，流程在日常的运行过程中也会有变化。iS-RPM 可以通过对业务数据的识别分析出业务的不同执行方式，将每个案例在整个周期中所采取的每一步叠加到数据模型中，还原出100%客观的流程视图，真实直观的展示员工每一次业务执行轨迹。', additional_kwargs={}, example=False),
  HumanMessage(content='请介绍一个RPM产品的实际案例 请用中文回答。', additional_kwargs={}, example=False),
  AIMessage(content='iS-RPM是一个按时间顺序排列的事件序列，显示了案例从流程开始到结束所采取的所有不同路径。

In [11]:
chain.memory

ConversationSummaryBufferMemory(human_prefix='Human', ai_prefix='AI', llm=ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.5, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=2048), prompt=PromptTemplate(input_variables=['summary', 'new_lines'], output_parser=None, partial_variables={}, template='根据聊天内容增量式生成摘要，在原始摘要上附加新的摘要。\nEXAMPLE\n当前摘要: \nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\n新的聊天内容:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\n新的摘要:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because

In [3]:
from typing import List, Optional
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoModel, AutoTokenizer


class ChatGLMService(LLM):
    max_token: int = 10000
    temperature: float = 0.1
    top_p = 0.9
    history = []
    tokenizer: object = None
    model: object = None

    def __init__(self):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "ChatGLM"

    def _call(self,
              prompt: str,
              stop: Optional[List[str]] = None) -> str:
        response, _ = self.model.chat(
            self.tokenizer,
            prompt,
            history=self.history,
            max_length=self.max_token,
            temperature=self.temperature,
        )
        if stop is not None:
            response = enforce_stop_tokens(response, stop)
        self.history = self.history + [[None, response]]
        return response

    def load_model(self, model_name_or_path: str = "THUDM/chatglm-6b"):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path,
            trust_remote_code=True
        )
        print('加载chatglm-6b', model_name_or_path)
        # 这里使用的是 量化后的模型int4
        self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).float().half().cuda()
        #.half().quantize(4).cuda()
        #.half().cuda()
        self.model=self.model.eval()